# Грунина Маргарита
# Вариант 7

$\displaystyle -\left( \frac{1}{2+x/3} u' \right)' + (e^{x/5})u = \lambda u$

$\displaystyle u'(-1) - 0.7u(-1) = u'(1) + 0.75u(1) = 0$

In [1]:
import numpy as np
import pandas as pd
import math
from scipy.special import jacobi

In [2]:
# краевые условия
alpha_1 = -0.7
alpha_2 = -1
beta_1 = 0.75
beta_2 = 1
# число координатных функций
n = 7

def p(x): return 1.0 / (2 + x / 3.0)
def q(x): return math.exp(x / 5.0)

In [8]:
# Вычисление интеграла по СКФ Симпсона
def Integrate(a, b, func, m):
    h = (b - a) / m
    alpha = a + h / 2
    beta = a + h
    return h / 6 * (func(a) + 4 * sum(func(alpha + k * h) for k in range(m)) +
                    func(b) + 2 * sum(func(beta + k * h) for k in range(m - 1)))

# Первая производная с помощью симметрической разности
def d(f):
    eps = 1e-10
    return lambda x: (f(x + eps) - f(x-eps)) / (2 * eps)

# Вторая производная с помощью симметрической разности
def dd(f):
    eps = 1e-5
    return lambda x: (f(x - eps) - 2 * f(x) + f(x+eps)) / eps**2

# Значение многочлена Лежандра степени i в точке x
def Omega(x, i): return math.sqrt((2 * i - 1) / 2.0) * jacobi(n = i - 1, alpha = 0, beta = 0)(x)

# Построение многочлена Лежандра степени i
def Omega_i(i): return lambda x: Omega(x, i)

# Скалярное произведение в L_2(-1, 1)
def Dot(y, z, n = 200):
    return Integrate(-1, 1, lambda x: y(x) * z(x), n)

# Норма в L_2(-1, 1)
def Norm(f):
    return math.sqrt(Dot(f, f))

# Энергетическое скалярное произведение в H_L для краевой задачи типа III
def BilinearForm(y, z):

    # Вычисление подынтегральной функции
    def UnderIntergal(x):
        return p(x) * d(y)(x) * d(z)(x) + q(x) * y(x) * z(x)

    return Integrate(-1, 1, UnderIntergal, 200) + alpha_1/alpha_2 * p(-1) * y(-1) * z(-1) + beta_1/beta_2 * p(1) * y(1) * z(1)

In [9]:
x = np.linspace(-1, 1, 1000)
p_max = max(map(p, x))
p_min = min(map(p, x))
q_max = max(map(q, x))
q_min = min(map(q, x))

# Получены подстановкой y = C1*cos(nu*x) + C2*sin(nu*x) в граничные условия и det = 0
nu1 = 0.760936
nu2 = 1.9300737

# Получение нижней и верхней границ по формуле
l_1_min = nu1**2 * p_min + q_min
l_1_max = nu1**2 * p_max + q_max
l_2_min = nu2**2 * p_min + q_min
l_2_max = nu2**2 * p_max + q_max

# Вычисляет нормированные собственные функции
def Calculate(nu):
    C = (beta_2 * nu * math.sin(nu) - beta_1 * math.cos(nu)) / (beta_1 * math.sin(nu) + beta_2 * nu * math.cos(nu))

    def eig_f(x): return math.cos(nu*x) + C * math.sin(nu*x)

    norm_eig_f = lambda x: eig_f(x) / Norm (eig_f)

    return norm_eig_f

# Метод Ритца
gamma_L = np.zeros((n, n))
for i in range(1, n+1):
    for j in range(1, n+1):
        gamma_L[i-1, j-1] = BilinearForm(Omega_i(i), Omega_i(j))

eigs_unsorted = list(np.linalg.eig(gamma_L))
eigs_unsorted[1] = eigs_unsorted[1].T
eigs = []
for i in range(n):
    eigs.append([eigs_unsorted[0][i], eigs_unsorted[1][i]])
eigs.sort()

for i in range(n):
    eigs[i].append(lambda x: sum([eigs[i][1][j] * Omega(x, j+1) for j in range(0, n)]))

Оценки на собственные числа

In [10]:
# В качетсве "точных" значений были взяты результаты из метода Ритца
print(l_1_min, "< 𝜆_1 <", l_1_max)
print(l_2_min, "< 𝜆_2 <", l_2_max)
data = {' ': ['min', 'max'],
        'оценка 𝜆_1': [l_1_min, l_1_max],
        'невязка 𝜆_1': [l_1_min - eigs[0][0], l_1_max - eigs[0][0]],
        'оценка 𝜆_2': [l_2_min , l_2_max],
        'невязка 𝜆_2': [l_2_min - eigs[1][0] , l_2_max - eigs[1][0]]}
pd.DataFrame(data)

1.0668837228334103 < 𝜆_1 < 1.5688169158177698
2.415238390548706 < 𝜆_2 < 3.456513450619184


,,оценка 𝜆_1,невязка 𝜆_1,оценка 𝜆_2,невязка 𝜆_2
0,min,1.066884,-0.232029,2.415238,-0.472708
1,max,1.568817,0.269904,3.456513,0.568567


Приближение первых двух собственных чисел по форуле: $λ_i = \frac{[y_i, y_i]}{(y_i, y_i)}$

In [12]:
# В качетсве "точных" значений были взяты результаты из метода Ритца
norm_eig_f_1 = Calculate(nu1)
norm_eig_f_2 = Calculate(nu2)

print('𝜆_1 =', BilinearForm(norm_eig_f_1, norm_eig_f_1))
print('𝜆_2 =', BilinearForm(norm_eig_f_2, norm_eig_f_2))

data_1 = {'𝜆_1 точное': [eigs[0][0]],
        '𝜆_1': [BilinearForm(norm_eig_f_1, norm_eig_f_1)],
        'невязка 𝜆_1': [BilinearForm(norm_eig_f_1, norm_eig_f_1) - eigs[0][0]],
        '𝜆_2 точное': [eigs[1][0]],
        '𝜆_2': [BilinearForm(norm_eig_f_2, norm_eig_f_2)],
        'невязка 𝜆_2': [BilinearForm(norm_eig_f_2, norm_eig_f_2) - eigs[1][0]]}

pd.DataFrame(data_1)

𝜆_1 = 1.3005509562747357
𝜆_2 = 2.8909176637763996


,𝜆_1 точное,𝜆_1,невязка 𝜆_1,𝜆_2 точное,𝜆_2,невязка 𝜆_2
0,1.298913,1.300551,0.001638,2.887946,2.890918,0.002971


Метод обратных итераций

In [13]:
columns = ['n', '𝜆(n)', '𝜆_diff', 'L*y-𝜆*y']
data = []  # Создаем пустой список для хранения данных строк таблицы

lamb1_ex = eigs[0][0]

# Метод скалярных произведений
for k in range(3, n+1):
    G = np.linalg.inv(gamma_L[:k,:k])
    x0 = 1.0 / k * np.ones((k, 1))
    accuracy = 10
    for i in range(accuracy):
        x1 = np.dot(G, x0)
        lamb = np.linalg.norm(x1, 2) / np.linalg.norm(x0, 2)
        x1 = x1 / np.linalg.norm(x1, 2)
        x0 = x1
    eigva = 1/lamb
    eig_f = lambda x: sum([x0[j] * omega(x, j+1) for j in range(0, k)])
    u = eig_f
    right = lambda x: eigva * u(x)
    left = lambda x: -(d(p)(x) * d(u)(x) + p(x) * dd(u)(x)) + q(x)*u(x)
    diff = lambda x: left(x) - right(x)

    # Создаем словарь для текущей строки таблицы и добавляем его в список данных
    row = {'n': k, '𝜆(n)': eigva, '𝜆_diff': eigva - lamb1_ex, 'L*y-𝜆*y': right(0) - left(0)}
    data.append(row)

# Создаем DataFrame из списка данных
table = pd.DataFrame(data, columns=columns)


In [14]:
table

,n,𝜆(n),𝜆_diff,L*y-𝜆*y
0,3,1.299548,6.349610e-04,[0.017980584826855495]
1,4,1.299032,1.193353e-04,[0.021109956917831907]
2,5,1.298915,2.287504e-06,[0.0003674973471292642]
3,6,1.298913,3.372967e-07,[0.0007063461297447393]
4,7,1.298913,2.830311e-07,[-2.024086780405554e-07]
